In [ ]:
!pip install git+https://github.com/uma-pi1/kge.git

  Cloning https://github.com/uma-pi1/kge.git to /tmp/pip-req-build-b7tsi249
  Running command git clone -q https://github.com/uma-pi1/kge.git /tmp/pip-req-build-b7tsi249
     |████████████████████████████████| 14.8 MB 6.9 MB/s 
     |████████████████████████████████| 776.8 MB 16 kB/s 
     |████████████████████████████████| 690 kB 70.9 MB/s 
     |████████████████████████████████| 395 kB 71.8 MB/s 
     |████████████████████████████████| 492 kB 63.2 MB/s 
     |████████████████████████████████| 3.6 MB 39.1 MB/s 
     |████████████████████████████████| 18.3 MB 15.4 MB/s 
  Created wheel for libkge: filename=libkge-0.1-py3-none-any.whl size=25613 sha256=6342802297067667ca01e9460abf89d0bd7f507d9485a5dbe0d9814943f2f759
  Stored in directory: /tmp/pip-ephem-wheel-cache-ax2kmzpk/wheels/cd/4f/1d/3a5ecce4ba442b99aed1da672be3bacce27bb2f578668ec193
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=2d4ff84ad9e345aa9b2a9db1bfe48db98effcb1b49efe77c707f2fa640100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/kge

Mounted at /content/drive
/content/drive/MyDrive/kge


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/kge"

In [ ]:
! echo $PYTHONPATH

/env/python:/content/drive/MyDrive/kge:/content/drive/MyDrive/kge


In [ ]:
! kge start 

Loading configuration /content/drive/MyDrive/kge/examples/toy-complex-train.yaml...
Using folder: /content/drive/MyDrive/kge/local/experiments/20220410-161246-toy-complex-train
Loading configuration of dataset toy from /content/drive/MyDrive/kge/data/toy ...
Loaded 280 keys from map entity_ids
Loaded 112 keys from map relation_ids
Loaded 4565 train triples
Loaded 109 valid triples
Loaded 152 test triples
[3b7a3cdb] Initializing 1-to-N training job...
[3b7a3cdb] Configuration:
[3b7a3cdb]   1vsAll: {class_name: TrainingJob1vsAll}
[3b7a3cdb]   KvsAll:
[3b7a3cdb]     class_name: TrainingJobKvsAll
[3b7a3cdb]     label_smoothing: 0.0
[3b7a3cdb]     query_types: {_po: true, s_o: false, sp_: true}
[3b7a3cdb]   ax_search:
[3b7a3cdb]     class_name: AxSearchJob
[3b7a3cdb]     num_sobol_trials: -1
[3b7a3cdb]     num_trials: 10
[3b7a3cdb]     parameter_constraints: []
[3b7a3cdb]     parameters: []
[3b7a3cdb]     sobol_seed: 0
[3b7a3cdb]   complex:
[3b7a3cdb]     class_name: ComplEx
[3b7a3cdb]     

In [ ]:
# Inspect data
### How do train/test/valid dataset look like?
### Are valid & test entities a subset of enetities in train?

In [ ]:
import pandas as pd
import numpy as np

for data in ['train', 'test', 'valid']:
    df = pd.read_csv(f'./data/toy/{data}.txt', header=None, sep='\t')
    df.columns = ['h', 'l', 't']
    globals()[data] = df.copy()

train_entity = set(train['h'].unique()).union(set(train['t'].unique()))
train.head()

,h,l,t
0,/m/0342h,/music/performance_role/regular_performances./...,/m/0l14qv
1,/m/02sgy,/music/performance_role/guest_performances./mu...,/m/0l14md
2,/m/0154j,/olympics/olympic_participating_country/athlet...,/m/09n48
3,/m/05v8c,/base/aareas/schema/administrative_area/admini...,/m/02j71
4,/m/040njc,/award/award_category/nominees./award/award_no...,/m/017gl1


In [ ]:
### Are valid entities a subset of enetities in train?
valid['h'].isin(train_entity).all(), valid['t'].isin(train_entity).all()

(True, True)

In [ ]:
### Are test entities a subset of enetities in train?
test['h'].isin(train_entity).all(), test['t'].isin(train_entity).all()

(True, True)

In [ ]:
### Are valid and test relations subsets of relations in train?
train_relation = set(train['l'].unique())
valid['l'].isin(train_relation).all(), test['l'].isin(train_relation).all()

(True, True)

# **Load and Subset Amazon Review Data**

Run Once

In [ ]:
full = pd.read_csv('./data/amazon_review/orig/full.txt', header=None, sep='\t')
full.columns = ['h', 'l', 't']

In [ ]:
rat = full.loc[full['l'].isin(['ranked_5', 'ranked_4', 'ranked_2', 'ranked_1'])].copy()
rat.loc[rat['l'].isin(['ranked_5', 'ranked_4']), 'l'] = 'likes'
rat.loc[rat['l'].isin(['ranked_2', 'ranked_1']), 'l'] = 'dislikes'
full = pd.concat([full, rat], axis=0).reset_index(drop=True)

In [ ]:
full['user_appearance'] = full.groupby(['h'])['h'].transform('size')
full.loc[~full['l'].isin(['has_reviewed', 'likes', 'dislikes']), 'user_appearance'] = np.nan

full['item_appearance'] = full.groupby(['t'])['t'].transform('size')
full.loc[~full['l'].isin(['has_reviewed', 'likes', 'dislikes']), 'item_appearance'] = np.nan

In [ ]:
df = full.loc[((full['l'].isin(['has_reviewed', 'likes', 'dislikes'])) & (full['user_appearance'] > 5) & (full['item_appearance'] > 5)) | (full['l'].isin(['also_buy', 'is_an_instance_of']))].copy().reset_index(drop=True)
df = df.loc[df[['h', 'l', 't']].notnull().all(axis=1)]
df.head()

,h,l,t,user_appearance,item_appearance
0,B00L62HMQG,also_buy,B00CQ16CT4,NaN,NaN
1,B00630Z16O,is_an_instance_of,"Clothing, Shoes & Jewelry",NaN,NaN
2,B01HFWIKH0,also_buy,B0795VHRY1,NaN,NaN
3,B01C7XYUZ8,is_an_instance_of,Sunglasses,NaN,NaN
4,B01H2ET0UM,also_buy,B07KY12JYZ,NaN,NaN


In [ ]:
lik = df.loc[df['l'].isin(['likes', 'dislikes'])].copy()
lik['usrgrp_idx'] = lik.groupby(['h']).cumcount()+1
lik['itmgrp_idx'] = lik.groupby(['t']).cumcount()+1

df_test = lik.loc[(lik['usrgrp_idx'] > 3) & (lik['itmgrp_idx'] > 3)].sample(2000)
df_train = pd.concat([df.loc[~(df['l'].isin(['likes', 'dislikes']))], lik.loc[~lik.index.isin(df_test.index)]], axis=0)
df_train['h_count'] = df_train.groupby('h').cumcount()+1
df_train['t_count'] = df_train.groupby('t').cumcount()+1
df_valid = df_train.loc[(df_train['h_count'] > 1) & (df_train['t_count'] > 1)].sample(10000)
df_train = df_train.loc[~df_train.index.isin(df_valid.index)]

In [ ]:
# Simpler version to override above - bipartite settings

lik = df.loc[df['l'].isin(['likes', 'dislikes'])].copy()
lik['usr_count'] = lik.groupby(['h'])['h'].transform('size')
lik['itm_count'] = lik.groupby(['t'])['t'].transform('size')

lik = lik.loc[(lik['usr_count'] > 3) & (lik['itm_count'] > 3)].copy()
lik['usrgrp_idx'] = lik.groupby(['h']).cumcount()+1
lik['itmgrp_idx'] = lik.groupby(['t']).cumcount()+1

df_test = lik.loc[(lik['usrgrp_idx'] > 2) & (lik['itmgrp_idx'] > 2)].sample(1500)
df_train = lik.loc[~lik.index.isin(df_test.index)].copy()
df_valid = df_train.loc[(df_train['usrgrp_idx'] > 1) & (df_train['itmgrp_idx'] > 1)].sample(3000)
df_train = df_train.loc[~df_train.index.isin(df_valid.index)].copy()

In [ ]:
df_train.shape, df_valid.shape, df_test.shape

((10004, 9), (3000, 9), (1500, 9))

In [ ]:
# check condition
usr_list = set(df_train.loc[df_train['usrgrp_idx'].notnull(), 'h'].unique())
itm_list = set(df_train.loc[df_train['itmgrp_idx'].notnull(), 't'].unique())
df_test['h'].isin(usr_list).all(), df_test['t'].isin(itm_list).all()

(True, True)

In [ ]:
# Save data

df_train.to_csv('./data/amazon_review/train.txt', header=None, index=None, sep='\t')
df_valid.to_csv('./data/amazon_review/valid.txt', header=None, index=None, sep='\t')
df_test.to_csv('./data/amazon_review/test.txt', header=None, index=None, sep='\t')

with open('./data/amazon_review/entities.txt', 'w') as f:
    entities = set(df['h'].unique()).union(set(df['t'].unique()))
    f.write('\n'.join(entities))

with open('./data/amazon_review/relations.txt', 'w') as f:
    relations = set(df['l'].unique())
    f.write('\n'.join(relations))

    # !python data/preprocess/preprocess_default.py data/amazon_review

In [ ]:
!python data/preprocess/preprocess_default.py data/amazon_review

Preprocessing data/amazon_review...
Found 10004 triples in train.txt
Found 3000 triples in valid.txt
Found 1500 triples in test.txt
2 distinct relations
6888 distinct entities
Writing relation and entity map...
dataset: {files.entity_ids.filename: entity_ids.del, files.entity_ids.type: map, files.relation_ids.filename: relation_ids.del,
  files.relation_ids.type: map, files.test.filename: test.del, files.test.size: 1500,
  files.test.split_type: test, files.test.type: triples, files.test_without_unseen.filename: test_without_unseen.del,
  files.test_without_unseen.size: 1500, files.test_without_unseen.split_type: test,
  files.test_without_unseen.type: triples, files.train.filename: train.del, files.train.size: 10004,
  files.train.split_type: train, files.train.type: triples, files.train_sample.filename: train_sample.del,
  files.train_sample.size: 3000, files.train_sample.split_type: train, files.train_sample.type: triples,
  files.valid.filename: valid.del, files.valid.size: 3000, f

# **Test Run - Simpler Setting**

We do a test run with the following subset of the Amazon Review data:

*   Define relation "like" and "dislike" if rating is above (resp. below) 3.
*   Restrict samples to the like and dislike relation only. This implies that for each triplet, the head and tail entities must be a user and an item respectively.
*   Include only users and items with more than 2 occurence in like and dislike interactions.
*   There are 10004 training samples, 3000 validation samples, and 1500 test samples. We ensured that users and items sampled in validation and test sets are necessarily included in training.
*   We experiment with various models supported vy `kge`, including TransE, ComplEX, RESCAL.

## **Job YAML**

### **Run Jobs**

Here's a sample of the YAML file for TransE. Similar for other model implementation.

```
job.type: train
dataset.name: amazon_review

train:
  optimizer.default:
    type: Adagrad
    args:
      lr: 0.1
  max_epochs: 100
  lr_scheduler: ReduceLROnPlateau
  lr_scheduler_args:
    mode: max
    patience: 4

model: transe
lookup_embedder:
  dim: 300
  regularize_weight: 0.8e-6
  initialize: normal_
  initialize_args:
    normal_:
      mean: 0.0
      std: 0.1
```

### **Search Jobs**

```
job.type: search
search.type: ax

dataset.name: amazon_review
model: transe
valid.metric: mean_reciprocal_rank_filtered

ax_search:
  num_trials: 100
  num_sobol_trials: 10  # remaining trials are Bayesian
  parameters:
    - name: train.batch_size
      type: choice
      values: [64, 128, 256, 512]
    - name: train.optimizer_args.lr
      type: range
      bounds: [0.0003, 1.0]
    - name: lookup_embedder.dim
      type: choice
      values: [128, 256, 512, 1024]
    - name: train.type
      type: fixed
      value: negative_sampling
    - name: train.max_epochs
      type: fixed
      value: 100
```

In [ ]:
!kge start amazon_bipartite_job_transe.yaml --job.device cuda

Streaming output truncated to the last 5000 lines.
[8ca42400]   optimizer_time: 0.0074312686920166016
[8ca42400]   other_time: 0.028299808502197266
[8ca42400]   parent_job_id: 2fca5fc2-d6e5-4948-8d82-78a96e326d91
[8ca42400]   prepare_time: 0.01550745964050293
[8ca42400]   scope: epoch
[8ca42400]   size: 10004
[8ca42400]   split: train
[8ca42400]   timestamp: 1649612271.6805274
[8ca42400]   type: negative_sampling
[8ca42400] Finished epoch 64.
[8ca42400] Saving checkpoint to /content/drive/MyDrive/kge/local/experiments/20220410-161509-amazon_bipartite_job_transe/00097/checkpoint_00064.pt...
[8ca42400] Removing old checkpoint /content/drive/MyDrive/kge/local/experiments/20220410-161509-amazon_bipartite_job_transe/00097/checkpoint_00063.pt...
[8ca42400] Starting epoch 65...
[8ca42400]   avg_cost: 1.1639266811837352
[8ca42400]   avg_loss: 1.1639266811837352
[8ca42400]   avg_penalties: {}
[8ca42400]   avg_penalty: 0.0
[8ca42400]   backward_time: 0.060448646545410156
[8ca42400]   batches: 40

In [ ]:
! kge dump trace /content/drive/MyDrive/kge/local/experiments/20220410-161509-amazon_bipartite_job_transe

job_id,dataset,model,reciprocal,job,job_type,split,epoch,avg_loss,avg_penalty,avg_cost,metric_name,metric,child_folder,child_job_id
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,75,,,,mean_reciprocal_rank_filtered,0.038640726725260414,00000,c0703e1e
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,55,,,,mean_reciprocal_rank_filtered,0.04957854715983073,00001,4914660f
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,20,,,,mean_reciprocal_rank_filtered,0.028392481486002603,00002,b4c1a357
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,65,,,,mean_reciprocal_rank_filtered,0.05176493326822917,00003,e0886b02
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,45,,,,mean_reciprocal_rank_filtered,0.04115789540608724,00004,ed991c86
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid,20,,,,mean_reciprocal_rank_filtered,0.03803598022460938,00005,739b3e95
2fca5fc2,amazon_review,transe,0,search,negative_sampling,valid

In [ ]:
!kge test /content/drive/MyDrive/kge/local/experiments/20220410-161509-amazon_bipartite_job_transe

In [ ]:
!kge start amazon_bipartite_job_complex.yaml --job.device cuda

Streaming output truncated to the last 5000 lines.
[3204ae67]   split: train
[3204ae67]   timestamp: 1649619122.7795799
[3204ae67]   type: 1vsAll
[3204ae67] Finished epoch 34.
[3204ae67] Saving checkpoint to /content/drive/MyDrive/kge/local/experiments/20220410-181354-amazon_bipartite_job_complex/00097/checkpoint_00034.pt...
[3204ae67] Removing old checkpoint /content/drive/MyDrive/kge/local/experiments/20220410-181354-amazon_bipartite_job_complex/00097/checkpoint_00033.pt...
[3204ae67] Starting epoch 35...
[3204ae67]   avg_cost: 4.1476326662271035
[3204ae67]   avg_loss: 4.1476326662271035
[3204ae67]   avg_penalties: {}
[3204ae67]   avg_penalty: 0.0
[3204ae67]   backward_time: 0.2679750919342041
[3204ae67]   batches: 157
[3204ae67]   entry_id: fb1e0145-1e04-457c-8e8c-e84c182ca0e9
[3204ae67]   epoch: 35
[3204ae67]   epoch_time: 0.5327107906341553
[3204ae67]   event: epoch_completed
[3204ae67]   forward_time: 0.1879100799560547
[3204ae67]   job: train
[3204ae67]   job_id: 3204ae67-f244-4